<a href="https://colab.research.google.com/github/stuckic/posCimatec/blob/main/20230830_Atividade_Altura_Peso_Genero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Passo 1: Carregamento do csv
uploaded = files.upload()

In [ ]:
# Passo 2: Ler os dados usando o pandas
file_name = next(iter(uploaded))

# file_name = f'D:\Downloads\Altura, Peso, Gênero.csv'
data = pd.read_csv(file_name, sep=',')

In [ ]:
# Label
data['Label'] = data.iloc[:, 3].apply(lambda x: 1 if x == 'Masculino' else 0)

# Correções do data set
data.iloc[:, 1:3] = data.iloc[:, 1:3].replace('175', '1,75', regex= True)
data.iloc[:, 1:3] = data.iloc[:, 1:3].replace(',', '.', regex= True)
data.iloc[:, 1:3] = data.iloc[:, 1:3].replace('kg', '', regex= True)

# Numeralização
data.iloc[:, 1:3] = data.iloc[:, 1:3].astype(float)

In [ ]:
# Mapear rótulos para cores
label_to_color = {
    'Feminino': 'red',
    'Masculino': 'blue'
}

# Criar uma lista de cores com base nos rótulos
point_colors = [label_to_color[label] for label in data.iloc[:, 3]]

# Criar um scatter plot usando os dados do DataFrame e as cores mapeadas
x1 = data.iloc[:, 1]
x2 = data.iloc[:, 2]

# Criar um scatter plot usando os dados do DataFrame e as cores mapeadas
scatter = plt.scatter(x1, x2, c=point_colors, cmap='coolwarm')

# Criar artistas para a legenda
legend_labels = list(label_to_color.keys())  # Usar as chaves do dicionário como rótulos da legenda
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=label_to_color[label], markersize=10, label=label) for label in legend_labels]

# Adicionar a legenda
plt.legend(handles=legend_handles)

# Adicionar rótulos e título
plt.xlabel('Altura')
plt.ylabel('Peso')
plt.title('Gráfico de Dispersão')

# Mostrar o gráfico
plt.show()

In [ ]:
# Balanceamento
categorias_list = list(data.iloc[:, 3].value_counts().index)
valores_list = list(data.iloc[:, 3].value_counts().values)
total = sum(valores_list)
for i in range(len(categorias_list)):
    print(f'{categorias_list[i]}: {valores_list[i]} - {round(valores_list[i]/total*100, 1)}%')

In [ ]:
# Bibliotecas do SVM
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Separação dos dados em X e y
X = data.iloc[:, 1:3].values
y = data.iloc[:, 4].values

In [ ]:
# Teste e treino
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, test_size=0.2, random_state=9)

In [ ]:
# Normalização
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train0)
X_test = scaler.transform(X_test0)
y_train = y_train0
y_test  = y_test0

## Regressão Logística

In [ ]:
# Regressão logística padrão
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

# Predição dos dados de test
y_pred = model_lr.predict(X_test)

In [ ]:
# Cálculo da acurácia
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Nova instância
x_new = scaler.transform(np.array([[1.74, 80]]))
x_new

In [ ]:
# Predição nova instância
model_lr.predict(x_new)[0]

## KNN

In [ ]:
# Criação da instância
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)

# Predição dos dados de test
y_pred = model_knn.predict(X_test)

In [ ]:
# Cálculo da acurácia
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Nova instância
x_new = scaler.transform(np.array([[1.74, 80]]))
x_new

In [ ]:
# Predição nova instância
model_knn.predict(x_new)[0]

In [ ]:
def plot_decision_boundary(estimator, X, y):
    X_color = X
    y_color = np.vectorize(lambda r: 'red' if r == 0 else 'blue')(y)
    x_axis, y_axis = np.arange(-.5, 1.5, .005), np.arange(-.5, 1.5, .005)
    xx, yy = np.meshgrid(x_axis, y_axis)
    xx_ravel = xx.ravel()
    yy_ravel = yy.ravel()
    X_grid = pd.DataFrame([xx_ravel, yy_ravel]).T
    y_grid_predictions = estimator.predict(X_grid)
    y_grid_predictions = y_grid_predictions.reshape(xx.shape)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.contourf(xx, yy, y_grid_predictions, cmap=plt.cm.autumn_r, alpha=.3)
    ax.scatter(X_color[:, 0], X_color[:, 1], color=y_color, alpha=1)
    ax.set(
        xlabel= 'Altura',
        ylabel= 'Peso',
        title=str(estimator))

In [ ]:
# Chart com os dados de Treino - lr
plot_decision_boundary(model_lr, X_train, y_train)

In [ ]:
# Chart com os dados de Treino - knn
plot_decision_boundary(model_knn, X_train, y_train)

In [ ]:
# Chart com os dados de Test - lr
plot_decision_boundary(model_lr, X_test, y_test)

In [ ]:
# Chart com os dados de Test - Knn
plot_decision_boundary(model_knn, X_test, y_test)

## Multi Layer Perceptron

In [ ]:
# Bibliotecas
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
# Montagem do modelo
model_mlp = Sequential()

# Layer de tamanho 32 e 2 atributos (altura e peso)
model_mlp.add(Dense(8, input_shape=(2,), activation='tanh'))
model_mlp.add(Dense(8, activation='sigmoid'))
# Layer de saída
model_mlp.add(Dense(2, activation='softmax'))
model_mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 5
epochs = 100

history = model_mlp.fit(X_train, y_train,
                        batch_size= batch_size,
                        epochs= epochs,
                        verbose= 1,
                        validation_split= 0.2)

In [ ]:
# Plotando o historico do processo de treinamento
plt.figure(figsize=(20, 5))
plt.plot(history.history['loss'], color='blue')
plt.plot(history.history['val_loss'], color='red')
plt.title('Model loss', fontsize=20)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Treinamento', 'Validação'], loc='upper right', fontsize=14)
plt.show()

In [ ]:
# Avaliação
score = model_mlp.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Nova instância
x_new = scaler.transform(np.array([[1.74, 80]]))
x_new

In [ ]:
# Predição nova instância
result = model_mlp.predict(x_new)
result

In [ ]:
# Resultado é o índice do vetor com o valor maior
np.argmax(result)

In [ ]:
np.array(np.argmax(result))

In [ ]:
# Adaptação do predict

class Onehot2Int(object):

    def __init__(self, model):
        self.model = model
    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.array(np.argmax(y_pred, axis=1))

model_mlp1=Onehot2Int(model_mlp)

In [ ]:
# Chart com os dados de Train - MLP
plot_decision_boundary(model_mlp1, X_train, y_train)

In [ ]:
# Chart com os dados de Test - MLP
plot_decision_boundary(model_mlp1, X_test, y_test)

## Trabalho

Repita o mesmo procedimento feito até aqui para o Perceptron Simples criado por vocês na atividade anterior. Implemente uma classe com os métodos fit, predict e evaluate (apenas para a acurácia).

Tente melhorar o desempenho dos modelos ajustando os hiperparâmetros. No caso do MLP, procure fazer com que não haja overfitting/underfitting, enquanto mantém a acurácia mais alta possível, e o loss mais baixo possível (dilema viés-variância).

Ao final, compare os resultados de todos os modelos, usando uma tabela para resumir o que encontrou. Discuta os resultados analisando os pontos fortes e fracos de cada um.